In [18]:
# !pip uninstall httpx httpcore
# !pip install --upgrade httpx httpcore

In [38]:
import numpy as np
import pandas as pd
import os
from openai import OpenAI
import math
import json
import re
import builtins
import csv
from deep_translator import GoogleTranslator

In [39]:
with open('config.json') as config_file:
    config = json.load(config_file)
    open_api_key = config['open_api_key']
client = OpenAI(api_key=open_api_key)

In [42]:
selected_files = ['100-200.csv', '0-100.csv', '200-300.csv', '300-400.csv', '700-800.csv', '900-1000.csv', '500-600.csv', '400-500.csv', '600-700.csv', '800-900.csv']
# selected_files += ['1000-1250.csv', '1250-1500.csv','1500-1750.csv',
# '1750-2000.csv']
selected_files += ['2000-2250.csv','2250-2500.csv',
'2500-2750.csv',
'2750-3000.csv']
selected_files += ['3000-3250.csv', '3250-3500.csv']#,'3500-3750.csv','3750-4000.csv']

In [43]:
language = 'Spanish'
language_code = 'es'

In [44]:
def get_sets_to_do(post_directory):
    os.makedirs(os.path.join(language, 'Vocabulary', post_directory), exist_ok = True)
    donesets = os.listdir(os.path.join(language, 'Vocabulary', post_directory))
    print('Done:', donesets)
    if '.DS_Store' in donesets:
        donesets.remove('.DS_Store')
    allsets = os.listdir(os.path.join(language, 'Vocabulary', 'split_sets'))
    notdonesets = [i for i in allsets if not i in donesets]
    todosets = notdonesets#['900-1000.csv']
    if '.DS_Store' in donesets:
        todosets.remove('.DS_Store')
    print('To Do:', todosets)
    return todosets
# todosets = get_sets_to_do('Split_Sets','ChatGPT_Sets')

In [45]:
frequencydf = pd.read_csv(os.path.join('Frequency Lists', language, 'Clean.csv'))
# frequencydf

Clean

VOCABULARY SPLIT

In [46]:
import os
import math

def split_into_studysets(frequencydf, language, initial_splitsize=100, new_splitsize=250):
    os.makedirs(os.path.join(language, 'Vocabulary', 'Split_Sets'), exist_ok=True)
    columns = [language]
    if 'POS' in frequencydf.columns:
        columns.append('POS')
    if 'Translation' in frequencydf.columns:
        columns.append('Translation')
    if 'English' in frequencydf.columns:
        frequencydf.rename(columns={'English': 'Translation'}, inplace=True)
        columns.append('Translation')
    if 'lemma' in frequencydf.columns:
        columns.append('show')
        columns.append('lemma')
    print('columns:', columns)
    
    total_rows = len(frequencydf)
    i = 0
    splitsize = initial_splitsize
    
    while i * splitsize < total_rows:
        if i * splitsize >= 1000:
            splitsize = new_splitsize
            start_index = 1000 + (i - 10) * new_splitsize  # Adjust start index for new splitsize
        else:
            start_index = i * splitsize
        
        end_index = min(start_index + splitsize, total_rows)
        subdf = frequencydf[start_index:end_index][columns]
        subdf.to_csv(language + '/Vocabulary/Split_Sets/' + str(start_index) + '-' + str(end_index) + '.csv')
        
        i += 1
    
    remainder = total_rows % splitsize
    print('remainder=', remainder)
    if remainder > 0:
        remaindersubdf = frequencydf[-remainder:][columns]
        remaindersubdf.to_csv(language + '/Vocabulary/Split_Sets/' + 'Remainder.csv')

    return subdf

# Example usage
df = split_into_studysets(frequencydf, language=language)


columns: ['Spanish', 'show', 'lemma']
remainder= 9


If you only want to do a subset of files for time purposes

In [47]:
def move_unselected_files(language, selected_files):
    # Create directory for spare sets
    spare_split_sets_dir = os.path.join(language, 'Vocabulary', 'Spare_Split_Sets')
    os.makedirs(spare_split_sets_dir, exist_ok=True)

    split_sets_dir = os.path.join(language, 'Vocabulary', 'Split_Sets')
    all_files = os.listdir(split_sets_dir)
    selected_files_set = set(selected_files)

    # Move files not in selected_files to the spare sets directory
    for file_name in all_files:
        if file_name not in selected_files_set:
            os.rename(
                os.path.join(split_sets_dir, file_name),
                os.path.join(spare_split_sets_dir, file_name)
            )
  # example of selected files
move_unselected_files(language, selected_files)

TypeError: 'str' object is not callable

CHATGPT

In [16]:
CHATGPT_todosets = get_sets_to_do('ChatGPT_Sets')

Done: ['1100-1200.csv', '1700-1800.csv', '1800-1900.csv', '1600-1700.csv', '1400-1500.csv', '2750-3000.csv', '1300-1400.csv', '2500-2750.csv', '1900-2000.csv', '2250-2500.csv', '1500-1600.csv', '1200-1300.csv', '2000-2250.csv', '1000-1100.csv']
To Do: ['800-900.csv', '100-200.csv', '600-700.csv', '700-800.csv', '0-100.csv', '300-400.csv', '900-1000.csv', '400-500.csv', '500-600.csv', '200-300.csv']


In [17]:
pd.isna(frequencydf.lemma[0])

False

In [18]:
def generate_sentence_in_target_language(variable_word, words_to_include, model = 'gpt-4o-2024-05-13', POS = 'None', tenses = [], lemma = 'None'):
    """
    Generate a sentence in lang using a specific word and part of speech, with words from a limited vocabulary, along with its English translation.

    Parameters:
    variable_word (str): The word to include in the sentence.
    POS (str): The part of speech the word should operate as.
    words_to_include (tuple): A list of words to try to include, and how many
    model (str): The model to use for generating the sentence (default is 'gpt-4o-2024-05-13').
    # tenses (list)

    Returns:
    str: A sentence in lang and its English translation, separated by a newline.
    """
    words_to_include = list(words_to_include[0].sample(words_to_include[1], replace=False))
    print(variable_word, words_to_include)
    if POS == 'None':
        if lemma != 'None':
            if not pd.isna(lemma):
                print(lemma)
                prompt = (
                      # f"Use only the following Russian words: {vocabulary_str}. "
                
                      f"Create a simple sentence in f{language} using basic vocabulary and containing the word '{variable_word}' "
                      f"({lemma}). Also, provide the English translation of the sentence separated by a newline."
                      f"Try to include the following words in the sentence: {words_to_include}"
                      )
            else:
                print('nolemma')
                prompt= (f"Create a simple sentence in f{language} using basic vocabulary and containing the word '{variable_word}' "
                ". Also, provide the English translation of the sentence separated by a newline."
                f"Try to include the following words in the sentence: {words_to_include}"
                         )
        
    else:
        print(POS)
        prompt = (
              f"Create a simple sentence in f{language} using basic vocabulary and containing the word '{variable_word}' "
              f"operating as a {POS} part of speech. Also, provide the English translation of the sentence separated by a newline."
              f"Try to include the following words in the sentence: {words_to_include}"
              # f"Use only the following tenses: {tenses}. "
              )
        
        if len(tenses) > 0:
            prompt += f"\n use only the following tenses: {tenses}"

    try:
        # Make the API request
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=50,
            temperature=0.2,
        )

        # Extract and return the output
        output_message = response.choices[0].message.content.strip()
        return output_message

    except Exception as e:
        return f"An error occurred: {str(e)}"

    # Extract and return the output
    output_message = response.choices[0].message.content.strip()
    return output_message

In [19]:
#TEST THE API
# frequencydf.columns
 
for index, row in frequencydf.iloc[304:308].iterrows():
    sentence = generate_sentence_in_target_language(row[language], (frequencydf.iloc[300:400][language], 2), lemma = row['lemma'])
    print(sentence)

suerte ['vete', 'noches']
suerte
Vete a dormir, que tengas buenas noches y mucha suerte.

Go to sleep, have a good night and good luck.
eh ['cuenta', 'capitán']
eh
¿Te diste cuenta, eh, capitán?
Did you notice, eh, captain?
rápido ['camino', 'caso']
rápido
El camino es rápido en este caso.
The road is quick in this case.
cuenta ['guerra', 'adiós']
contar
El soldado cuenta historias de la guerra antes de decir adiós.
The soldier tells stories of the war before saying goodbye.


In [20]:
def chatonset(s_set_df, set_name, n = 2):
    words_to_include = (s_set_df[language], n)
    s_set_df['ChatGPT_Sentence'] = s_set_df.apply(lambda row: generate_sentence_in_target_language(row[language], words_to_include, lemma = row['lemma']), axis = 1)
    s_set_df.to_csv(os.path.join(language, 'Vocabulary','ChatGPT_Sets', set_name))
    return s_set_df

In [21]:
def do_all_sets(post_directory, func, language = language):
    os.makedirs(os.path.join(language, 'Vocabulary', post_directory), exist_ok = True)
    allsets = os.listdir(os.path.join(language, 'Vocabulary', 'split_sets'))
    todosets = get_sets_to_do(post_directory)
    for s_set in todosets:
        if s_set[0] != '.':
        # # if s_set == '500-600.csv':
        #     # print(s_set)
            s_set_df = pd.read_csv(
                os.path.join(language, 'Vocabulary', 'split_sets', s_set)
            )
            print(s_set_df)
            print(s_set)
            func(s_set_df, s_set)
    return

In [22]:
do_all_sets('ChatGPT_Sets', chatonset)

Done: ['1100-1200.csv', '1700-1800.csv', '1800-1900.csv', '1600-1700.csv', '1400-1500.csv', '2750-3000.csv', '1300-1400.csv', '2500-2750.csv', '1900-2000.csv', '2250-2500.csv', '1500-1600.csv', '1200-1300.csv', '2000-2250.csv', '1000-1100.csv']
To Do: ['800-900.csv', '100-200.csv', '600-700.csv', '700-800.csv', '0-100.csv', '300-400.csv', '900-1000.csv', '400-500.csv', '500-600.csv', '200-300.csv']
    Unnamed: 0   Spanish                show     lemma
0          800  ejército            ejército  ejército
1          801    futuro              futuro    futuro
2          802    llevar              llevar    llevar
3          803  compañía            compañía  compañía
4          804    venido    *venido* (venir)     venir
..         ...       ...                 ...       ...
95         895     pagar               pagar     pagar
96         896     fotos      *fotos* (foto)      foto
97         897    centro              centro    centro
98         898    centro  *centro* (centrar)   c

ADD TRANSLATION INFORMATION:

In [23]:
def add_set_translation_information(s_set_df, set_name, code = language_code):
    chat = pd.read_csv(os.path.join(language, 'Vocabulary', 'ChatGPT_Sets', set_name))
    # Apply the translation
    if 'Translation' in chat.columns:
        if 'POS' in chat.columns:
            s_set_df = chat[[language, 'Translation']]
        
    else:
        s_set_df['Translation'] = s_set_df.apply(
            lambda row: GoogleTranslator(source= code, target='en').translate(row[language]), axis=1
        )

    # Save to CSV
    s_set_df.to_csv(os.path.join(language, 'Vocabulary/Translated Sets', set_name), index=False)
    return s_set_df

In [24]:
import spacy
nlp = spacy.load("es_core_news_sm")
def add_spacy_POS_info(s_set_filepath):
    s_set_df = pd.read_csv(s_set_filepath)
    if 'POS' in s_set_df.columns:
        pass
    else:
        # Custom mapping for more understandable POS descriptions
        pos_mapping = {
            'ADJ': 'Adjective',
            'ADP': 'Preposition',
            'ADV': 'Adverb',
            'AUX': 'Auxiliary',
            'CONJ': 'Conjunction',
            'CCONJ': 'Conjunction',
            'DET': 'Determiner',
            'INTJ': 'Interjection',
            'NOUN': 'Noun',
            'NUM': 'Numeral',
            'PART': 'Particle',
            'PRON': 'Pronoun',
            'PROPN': 'Proper noun',
            'PUNCT': 'Punctuation',
            'SCONJ': 'Conjunction',
            'SYM': 'Symbol',
            'VERB': 'Verb',
            'X': 'Other'
        }
        
        # Function to get POS, tag, and layman's description
        def get_pos_tag_layman(word):
            doc = nlp(word)
            pos, tag, layman_description = None, None, None
            for token in doc:
                pos = token.pos_
                tag = token.tag_
                layman_description = pos_mapping.get(pos, 'Unknown')
            return pos, tag, layman_description
        
        # Apply the function to each word in the DataFrame
        s_set_df[['SpaCy_POS', 'SpaCy_Tag', 'POS']] = s_set_df['Spanish'].apply(lambda word: pd.Series(get_pos_tag_layman(word)))
        s_set_df.to_csv(s_set_filepath)
    
    return s_set_df

In [25]:
translation_todosets = get_sets_to_do('Translated Sets')

Done: ['1100-1200.csv', '1700-1800.csv', '1800-1900.csv', '1600-1700.csv', '1400-1500.csv', '2750-3000.csv', '1300-1400.csv', '2500-2750.csv', '1900-2000.csv', '2250-2500.csv', '1500-1600.csv', '1200-1300.csv', '2000-2250.csv', '1000-1100.csv']
To Do: ['800-900.csv', '100-200.csv', '600-700.csv', '700-800.csv', '0-100.csv', '300-400.csv', '900-1000.csv', '400-500.csv', '500-600.csv', '200-300.csv']


In [26]:
for set in translation_todosets:
    print(set)
    do_all_sets('Translated Sets', add_set_translation_information)
for set in os.listdir(os.path.join(language, 'Vocabulary', 'Translated Sets')):
    add_spacy_POS_info(os.path.join(language, 'Vocabulary', 'Translated Sets', set))

800-900.csv
Done: ['1100-1200.csv', '1700-1800.csv', '1800-1900.csv', '1600-1700.csv', '1400-1500.csv', '2750-3000.csv', '1300-1400.csv', '2500-2750.csv', '1900-2000.csv', '2250-2500.csv', '1500-1600.csv', '1200-1300.csv', '2000-2250.csv', '1000-1100.csv']
To Do: ['800-900.csv', '100-200.csv', '600-700.csv', '700-800.csv', '0-100.csv', '300-400.csv', '900-1000.csv', '400-500.csv', '500-600.csv', '200-300.csv']
    Unnamed: 0   Spanish                show     lemma
0          800  ejército            ejército  ejército
1          801    futuro              futuro    futuro
2          802    llevar              llevar    llevar
3          803  compañía            compañía  compañía
4          804    venido    *venido* (venir)     venir
..         ...       ...                 ...       ...
95         895     pagar               pagar     pagar
96         896     fotos      *fotos* (foto)      foto
97         897    centro              centro    centro
98         898    centro  *centro* (

Make into Quizlet

In [34]:
def make_quizlet_set(set_name):
    df = pd.read_csv(os.path.join(language, 'Vocabulary', 'ChatGPT_Sets', set_name))
    transdf = pd.read_csv(os.path.join(language, 'Vocabulary', 'Translated Sets', set_name))

    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].apply(lambda x: re.sub(r'\n+', '\n', x))
    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.replace(r'\(see[^)]*\)', '', regex=True)
    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.strip()
    
    if 'show' in df.columns:
        df['quizlet'] = '*' + transdf['Translation'] + '*' + '\n' + df['ChatGPT_Sentence']
        df[language] = df['show']
    else:
        df['quizlet'] = df['POS'] + ' : ' + '*' + transdf['Translation'] + '*' + '\n' + df['ChatGPT_Sentence']

    df = df[[language, 'quizlet']]
    os.makedirs(os.path.join(language, 'Vocabulary', 'Quizlet Sets'), exist_ok=True)
    output_path = os.path.join(language, 'Vocabulary', 'Quizlet Sets', set_name.replace('.csv', '.txt'))

    # Saving the data to a tab-separated text file with 3 lines between each row
    with open(output_path, 'w', encoding='utf-8') as outfile:
        for index, row in df.iterrows():
            outfile.write(f"{row[language]}\t{row['quizlet']}\n\n\n")

    print(f"Text file saved to {output_path}")
    return df

In [35]:
def make_reverse_quizlet_set(set_name):
    df = pd.read_csv(os.path.join(language, 'Vocabulary', 'ChatGPT_Sets', set_name))
    transdf = pd.read_csv(os.path.join(language, 'Vocabulary', 'Translated Sets', set_name))

    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].apply(lambda x: re.sub(r'\n+', '\n', x))

    # Removing text within parentheses that start with 'see'
    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.replace(r'\(see[^)]*\)', '', regex=True).str.strip()
    
    df['quizlet_front'] = transdf['POS'] + ' : ' + '*' + transdf['Translation'] + '*'
    df['quizlet_back'] =  '*' + df[language] + '*' + '\n' + df['ChatGPT_Sentence']

    df = df[['quizlet_front', 'quizlet_back']]
    os.makedirs(os.path.join(language, 'Vocabulary', 'Reverse Quizlet Sets'), exist_ok=True)
    output_path = os.path.join(language, 'Vocabulary', 'Reverse Quizlet Sets', set_name.replace('.csv', '.txt'))

    # Saving the data to a tab-separated text file with 3 lines between each row
    with open(output_path, 'w', encoding='utf-8') as outfile:
        for index, row in df.iterrows():
            outfile.write(f"{row['quizlet_front']}\t{row['quizlet_back']}\n\n\n")

    print(f"Text file saved to {output_path}")
    return df

In [36]:
builtin_set = builtins.set
a = builtin_set(os.listdir(os.path.join(language, 'Vocabulary', 'ChatGPT_Sets')))
b = builtin_set(os.listdir(os.path.join(language, 'Vocabulary', 'Translated Sets')))
quizlet_todosets = a.intersection(b)
reverse_quizlet_todosets = quizlet_todosets

In [37]:
quizlet_todosets
for s_set in quizlet_todosets:
    make_quizlet_set(s_set)
for s_set in reverse_quizlet_todosets:
    make_reverse_quizlet_set(s_set)

Text file saved to Spanish/Vocabulary/Quizlet Sets/2750-3000.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/2000-2250.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/2500-2750.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/1400-1500.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/900-1000.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/1800-1900.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/600-700.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/1000-1100.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/700-800.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/400-500.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/1300-1400.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/1100-1200.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/1200-1300.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/200-300.txt
Text file saved to Spanish/Vocabulary/Quizlet Sets/500-600.txt
Text file saved to Spanish/Vocabular

In [31]:
# def makequizlet_step2(input_file):
#     output_file = input_file 
#     with open(input_file, 'r', newline='', encoding='utf-8') as infile:
#         reader = csv.reader(infile)
#         rows = list(reader)
# 
#     # Write the output CSV file with extra newlines
#     with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
#         writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)
#         for row in rows:
#             writer.writerow(row)
#             writer.writerow([])  # Add a blank row
#             writer.writerow([])  # Add a blank row
#             writer.writerow([])
# 
#     print(f"Modified CSV file saved as {output_file}")
#     
# for file in os.listdir(os.path.join(language, 'Vocabulary/Quizlet Sets')):
#     if file[0] != '.':
#         print(file)
#         makequizlet_step2(os.path.join(language, 'Vocabulary/Quizlet Sets/' + file))

In [32]:
# import csv
# def make_quizlet_set(set_name):
#     df = pd.read_csv(os.path.join(language, 'Vocabulary', 'ChatGPT_Sets', set_name))
#     transdf = pd.read_csv(os.path.join(language, 'Vocabulary', 'Translated Sets', set_name))
#     
#     df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].apply(lambda x: re.sub(r'\n+', '\n', x))
#     
#     # Removing text within parentheses that start with 'see'
#     df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.replace(r'\(see[^)]*\)', '', regex=True)   
#     # Removing any extra spaces that might result from the replacement
#     df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.strip()
#     df['quizlet'] = transdf['POS'] + ' : ' + '*' + transdf['Translation'] + '*' + '\n\n' + df['ChatGPT_Sentence']
# 
# 
#     df = df[[language, 'quizlet']]
#     os.makedirs(os.path.join(language, 'Vocabulary', 'Quizlet Sets'), exist_ok = True)
#     output_path = os.path.join(language, 'Vocabulary', 'Quizlet Sets', set_name)
#     df.to_csv(output_path, index=False, quoting=csv.QUOTE_MINIMAL)
#     # df.to_csv(output_path, index=False, quoting=csv.QUOTE_NONE, escapechar = ddd)
#     print(f"CSV file saved to {output_path}")
#     return df

In [33]:
# def remove_quotes_from_csv(file_path):
#     # Read the CSV file and remove quotes
#     with open(file_path, 'r', newline='') as infile:
#         reader = csv.reader(infile)
#         rows = [[cell.replace('"', '') for cell in row] for row in reader]
#     
#     # Write the cleaned data back to the CSV file
#     with open(file_path, 'w', newline='') as outfile:
#         writer = csv.writer(outfile)
#         writer.writerows(rows)

In [27]:
# for file in os.listdir('Russian/Vocabulary/Quizlet Sets'):
#     if file[0] != '.':
#         print(file)
#         makequizlet_step2('Russian/Vocabulary/Quizlet Sets/' + file)